In [1]:
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import aquarel as aq
from ineqpy.inequality import gini
import scipy.stats as st
import pyalex as alex
alex.config.email = "noah0roussel01980@gmail.com"
works = pl.read_csv("../data/works/works_q1.csv")
works = works.filter(pl.col("year") != 2025)

year_begin = 1920
year_end = 2024
works = works.with_columns(
    age=2025 - pl.col("year")
)


In [ ]:
cbc_per_year = (
    works
    .select(
        [pl.col("year"), pl.col("cited_by_count"), pl.col("title")]
    )
    .group_by(
        pl.col("year"),
        maintain_order=True
    )
    .agg(
        pl.col("cited_by_count").mean().name.prefix("mean_")
    )
) 

works = works.join(
    cbc_per_year,
    on="year",
    how="left"
) 

works = (
    works
    .with_columns(
        mncs = pl.col("cited_by_count") / pl.col("mean_cited_by_count")
    )
    .drop("mean_cited_by_count")
)
cols_authors = [col for col in works.columns if col.startswith("author_")]

works = works.with_columns(
    authors_count = sum(
        [pl.col(col).is_not_null().cast(pl.Int8) for col in cols_authors]
    )

)

works = works.with_columns(
    title = pl.col("title").fill_null("").str.to_lowercase(),
    abstract = pl.col("abstract").fill_null("").str.to_lowercase()
)

works = works.with_columns( 
    review = (
        pl.col("title").str.contains("review") |
        pl.col("abstract").str.contains("review")
    ),
    meta_analysis = (
        pl.col("title").str.contains("meta[\u00AD-]?analysis") |
        pl.col("abstract").str.contains("meta[\u00AD-]?analysis")
    )
)

works = works.with_columns([
    pl.when(pl.col("countries_distinct_count").is_null() | pl.col("countries_distinct_count").is_nan() | (pl.col("countries_distinct_count") == 0))
      .then(1)
      .otherwise(pl.col("countries_distinct_count"))
      .alias("countries_distinct_count")
])
works = works.with_columns([
    pl.when(pl.col("institutions_distinct_count").is_null() | pl.col("institutions_distinct_count").is_nan() | (pl.col("institutions_distinct_count") == 0))
      .then(1)
      .otherwise(pl.col("institutions_distinct_count"))
      .alias("institutions_distinct_count")
])

works

title,year,cited_by_count,countries_distinct_count,institutions_distinct_count,citation_normalized_percentile,primary_topic,keywords,concepts,referenced_works_count,referenced_works,abstract,abstract_inverted_index,journal,author_1,author_2,author_3,institution_1,institution_2,institution_3,country_1,country_2,country_3,cited_by_count_2025,cited_by_count_2024,cited_by_count_2023,cited_by_count_2022,cited_by_count_2021,cited_by_count_2020,cited_by_count_2019,cited_by_count_2018,cited_by_count_2017,cited_by_count_2016,cited_by_count_2015,cited_by_count_2014,cited_by_count_2013,cited_by_count_2012,…,country_239,country_240,country_241,country_242,country_243,country_244,country_245,country_246,country_247,country_248,country_249,country_250,country_251,country_252,country_253,country_254,country_255,country_256,country_257,country_258,country_259,country_260,country_261,country_262,country_263,country_264,country_265,country_266,country_267,country_268,country_269,country_270,age,mncs,authors_count,review,meta_analysis
str,i64,i64,i64,i64,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,f64,i8,bool,bool
"""ros stress in cancer cells and…",2004,1881,1,1,"""{'value': 0.816836, 'is_in_top…","""Redox biology and oxidative st…","""[{'id': 'https://openalex.org/…","""[{'id': 'https://openalex.org/…",158,"""['https://openalex.org/W116540…","""""",null,"""Drug Resistance Updates""","""https://openalex.org/A50102407…","""https://openalex.org/A50076048…","""https://openalex.org/A51006444…","""['The University of Texas MD A…","""['The University of Texas MD A…","""['The University of Texas MD A…","""US""","""US""","""US""",40.0,71.0,85.0,108.0,133.0,133.0,97.0,91.0,127.0,111.0,119.0,130.0,108.0,117.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,21,35.103559,3,false,false
"""aminoglycoside modifying enzym…",2010,1294,1,1,"""{'value': 0.995475, 'is_in_top…","""Bacteriophages and microbial i…","""[{'id': 'https://openalex.org/…","""[{'id': 'https://openalex.org/…",336,"""['https://openalex.org/W141030…","""""",null,"""Drug Resistance Updates""","""https://openalex.org/A50909767…","""https://openalex.org/A50055229…",null,"""['California State University,…","""['California State University,…",null,"""US""","""US""",null,74.0,108.0,123.0,144.0,136.0,162.0,108.0,92.0,72.0,70.0,57.0,52.0,45.0,26.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,15,24.857588,2,false,false
"""overcoming the blood–brain tum…",2015,898,2,6,"""{'value': 0.998282, 'is_in_top…","""Glioma Diagnosis and Treatment""","""[{'id': 'https://openalex.org/…","""[{'id': 'https://openalex.org/…",159,"""['https://openalex.org/W148541…","""""",null,"""Drug Resistance Updates""","""https://openalex.org/A50341518…","""https://openalex.org/A50913295…","""https://openalex.org/A50395930…","""['The Netherlands Cancer Insti…","""['Amsterdam UMC Location Vrije…","""['The Netherlands Cancer Insti…","""NL""","""NL""","""NL""",80.0,87.0,97.0,110.0,116.0,119.0,91.0,80.0,59.0,43.0,15.0,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,10,23.127161,6,false,false
"""targeting the pi3k/akt/mtor pa…",2008,786,1,2,"""{'value': 0.999673, 'is_in_top…","""PI3K/AKT/mTOR signaling in can…","""[{'id': 'https://openalex.org/…","""[{'id': 'https://openalex.org/…",236,"""['https://openalex.org/W150966…","""""",null,"""Drug Resistance Updates""","""https://openalex.org/A50898992…","""https://openalex.org/A50296122…","""https://openalex.org/A51080787…",

In [3]:
import polars as pl
from tqdm import tqdm

cols_authors = [f"author_{i}" for i in range(1, 21)]
institutions_col = [f"institution_{i}" for i in range(1, 193)]

works = works.sort("year").with_row_index("index_work")

years = works["year"].unique().to_list()
results = []

for y in tqdm(years, desc="Processing years"):
    batch = works.filter(pl.col("year") == y)

    authors_batch = (
        batch
        .select(["index_work", "title", "year", "cited_by_count", "mncs", *cols_authors])
        .unpivot(
            on=cols_authors,
            index=["title", "year", "index_work", "cited_by_count", "mncs"],
            variable_name="author_pos",
            value_name="author_name"
        )
        .filter(pl.col("author_name").is_not_null())
        .with_columns(count=pl.lit(1))
        .sort("index_work")
        .with_columns([
            (pl.col("count").cum_sum().over("author_name") - pl.col("count")).alias("cumulative_count"),
            (pl.col("cited_by_count").cum_sum().over("author_name") - pl.col("cited_by_count")).alias("cumulative_citations"),
            (pl.col("mncs").cum_sum().over("author_name") - pl.col("mncs")).alias("cumulative_mncs"),
        ])
        .with_columns(
            pl.when(pl.col("cumulative_count") == 0)
            .then(pl.lit(1))
            .otherwise(pl.col("cumulative_mncs") / pl.col("cumulative_count"))
            .alias("mean_past_mncs_authors")
        )
        .group_by("index_work")
        .agg([
            pl.mean("cumulative_count").alias("mean_past_contributions_authors"),
            pl.mean("mean_past_mncs_authors").alias("mean_past_mncs_authors")
        ])
    )

    institutions_batch = (
        batch
        .select(["index_work", "year", "cited_by_count", "mncs", *institutions_col])
        .unpivot(
            on=institutions_col,
            index=["year", "index_work", "cited_by_count", "mncs"],
            variable_name="institution_pos",
            value_name="institution_name"
        )
        .filter(pl.col("institution_name").is_not_null())
        .with_columns(
            count=pl.lit(1),
            institution_simple=pl.col("institution_name")
                .str.split("',", inclusive=False)
                .list.first()
                .str.slice(2)
        )
        .sort("index_work")
        .with_columns([
            (pl.col("count").cum_sum().over("institution_simple") - pl.col("count")).alias("cumulative_count"),
            (pl.col("cited_by_count").cum_sum().over("institution_simple") - pl.col("cited_by_count")).alias("cumulative_citations"),
            (pl.col("mncs").cum_sum().over("institution_simple") - pl.col("mncs")).alias("cumulative_mncs"),
        ])
        .with_columns(
            pl.when(pl.col("cumulative_count") == 0)
            .then(pl.lit(1))
            .otherwise(pl.col("cumulative_mncs") / pl.col("cumulative_count"))
            .alias("mean_past_mncs_institutions")
        )
        .group_by("index_work")
        .agg([
            pl.mean("cumulative_count").alias("mean_past_contributions_institutions"),
            pl.mean("mean_past_mncs_institutions").alias("mean_past_mncs_institutions")
        ])
    )

    batch_final = batch.join(authors_batch, on="index_work", how="left") \
                       .join(institutions_batch, on="index_work", how="left")

    results.append(batch_final)

works_final = pl.concat(results)

works_final = works_final.with_columns([
    pl.col("mean_past_mncs_authors").fill_null(1),
    pl.col("mean_past_contributions_authors").fill_null(0),
    pl.col("mean_past_mncs_institutions").fill_null(1),
    pl.col("mean_past_contributions_institutions").fill_null(0)
])

works = works_final


Processing years: 100%|██████████| 116/116 [00:25<00:00,  4.60it/s]


In [4]:
works = works.with_columns(
    primary_topic = pl.col("primary_topic").str.replace("'","", literal=True, n=2).str.replace("(","", literal=True).str.replace(")","", literal=True).str.replace(",","", literal=True),
    primary_subfield = pl.col("primary_subfield").str.replace("'","", literal=True, n=2).str.replace("(","", literal=True).str.replace(")","", literal=True).str.replace(",","", literal=True),
    primary_field = pl.col("primary_field").str.replace("'","", literal=True, n=2).str.replace("(","", literal=True).str.replace(")","", literal=True).str.replace(",","", literal=True), 
    primary_domain = pl.col("primary_domain").str.replace("'","", literal=True, n=2).str.replace("(","", literal=True).str.replace(")","", literal=True).str.replace(",","", literal=True)
)

In [5]:
works

index_work,title,year,cited_by_count,countries_distinct_count,institutions_distinct_count,citation_normalized_percentile,primary_topic,keywords,concepts,referenced_works_count,referenced_works,abstract,abstract_inverted_index,journal,author_1,author_2,author_3,institution_1,institution_2,institution_3,country_1,country_2,country_3,cited_by_count_2025,cited_by_count_2024,cited_by_count_2023,cited_by_count_2022,cited_by_count_2021,cited_by_count_2020,cited_by_count_2019,cited_by_count_2018,cited_by_count_2017,cited_by_count_2016,cited_by_count_2015,cited_by_count_2014,cited_by_count_2013,…,country_243,country_244,country_245,country_246,country_247,country_248,country_249,country_250,country_251,country_252,country_253,country_254,country_255,country_256,country_257,country_258,country_259,country_260,country_261,country_262,country_263,country_264,country_265,country_266,country_267,country_268,country_269,country_270,age,mncs,authors_count,review,meta_analysis,mean_past_contributions_authors,mean_past_mncs_authors,mean_past_contributions_institutions,mean_past_mncs_institutions
u32,str,i64,i64,i64,i64,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,f64,i8,bool,bool,f64,f64,f64,f64
0,"""on the pharmacological action …",1909,47,1,1,"""{'value': 0.99265, 'is_in_top_…","""Chemistry and Stereochemistry …","""[]""","""[{'id': 'https://openalex.org/…",0,"""[]""","""""",null,"""Journal of Pharmacology and Ex…","""https://openalex.org/A51102708…","""https://openalex.org/A50329477…",null,null,null,null,null,null,null,null,1.0,null,null,null,1.0,null,null,null,null,null,null,1.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,116,10.143885,2,false,false,0.0,1.0,0.0,1.0
1,"""the antagonism of the adrenal …",1909,15,1,1,"""{'value': 0.951659, 'is_in_top…","""Hormonal Regulation and Hypert…","""[]""","""[{'id': 'https://openalex.org/…",0,"""[]""","""""",null,"""Journal of Pharmacology and Ex…","""https://openalex.org/A51116416…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,116,3.23741,1,false,false,0.0,1.0,0.0,1.0
2,"""quantitative experiments with …",1909,14,1,1,"""{'value': 0.978142, 'is_in_top…","""Tuberculosis Research and Epid…","""[{'id': 'https://openalex.org/…","""[{'id': 'https://openalex.org/…",0,"""[]""","""""",null,"""Journal of Pharmacology and Ex…","""https://openalex.org/A50125106…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,116,3.021583,1,false,false,0.0,1.0,0.0,1.0
3,"""the comparative toxicity of th…",1909,12,1,1,"""{'value': 0.970958, 'is_in_top…","""Pharmacological Effects and To…","""[{'id': 'https://openalex.org/…","""[{'id': 'https://openalex.org/…",0,"""[]""","""""","""{'The': [0, 122], 'physiologic…","""Journal of Pharmacology and Ex…","""https://openalex.org/A51121624…","""https://openalex.org/A50802778…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,116,2.589928,2,false,false,0.0,1.0,0.0,1.0
4,"""on the relation between the to…",1909,11,1,1,"""{'value': 0.911704, 'is_in_top…","""Chemical Thermodynamics and Mo…","""[{'id': 'https://openalex.org/…","""[{'id': 'https://openalex.org/…",0,"""[]""","""""",null,"""Journal of Pharmacology and Ex…",""

In [6]:
works.lazy().collect().write_parquet(
    "../data/modelling/works_pre_topics.parquet",
    compression="zstd",
    row_group_size=2_000
)